<a href="https://colab.research.google.com/github/aryan0931/E-commerce-delivery/blob/main/SCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install scikit-learn tensorflow keras-tuner web3 eth-account py-solc-x eth-tester pandas numpy matplotlib seaborn ipython py-evm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from web3 import Web3
from eth_account import Account
from solcx import compile_source, install_solc
from datetime import datetime
import time
from eth_tester import EthereumTester, PyEVMBackend
from web3.providers.eth_tester import EthereumTesterProvider

# Install Solidity compiler
install_solc('0.8.0')

# Solidity smart contract with Proof of Green Compliance
CONTRACT_SOURCE = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract SupplyChain {
    struct Order {
        string orderId;
        string product;
        uint256 quantity;
        string status;
        uint256 timestamp;
        uint256 emissions; // in kg CO2 * 100 for precision
        address sender;
    }

    mapping(string => Order) public orders;
    uint256 public orderCount;
    uint256 public totalEmissions; // in kg CO2 * 100
    uint256 public emissionThreshold = 1000; // 10 kg CO2 * 100

    event OrderCreated(string orderId, string product, uint256 quantity, uint256 emissions);
    event ShipmentUpdated(string orderId, string status, uint256 emissions);
    event ReorderTriggered(string orderId, string product, uint256 quantity);

    function createOrder(string memory orderId, string memory product, uint256 quantity, uint256 emissions) public {
        require(emissions <= emissionThreshold, "Emissions exceed Proof of Green Compliance threshold");
        require(bytes(orders[orderId].orderId).length == 0, "Order ID already exists");

        orders[orderId] = Order(orderId, product, quantity, "Created", block.timestamp, emissions, msg.sender);
        orderCount++;
        totalEmissions += emissions;
        emit OrderCreated(orderId, product, quantity, emissions);
    }

    function updateShipment(string memory orderId, string memory status, uint256 emissions) public {
        require(bytes(orders[orderId].orderId).length != 0, "Order does not exist");
        require(emissions <= emissionThreshold, "Emissions exceed Proof of Green Compliance threshold");

        orders[orderId].status = status;
        orders[orderId].emissions += emissions;
        totalEmissions += emissions;
        emit ShipmentUpdated(orderId, status, emissions);
    }

    function automateReorder(string memory orderId, string memory product, uint256 currentQuantity, uint256 threshold, string memory newOrderId, uint256 emissions) public {
        require(currentQuantity < threshold, "Current quantity above threshold");
        createOrder(newOrderId, product, threshold - currentQuantity, emissions);
        emit ReorderTriggered(newOrderId, product, threshold - currentQuantity);
    }

    function getOrder(string memory orderId) public view returns (string memory, string memory, uint256, string memory, uint256, uint256, address) {
        Order memory order = orders[orderId];
        return (order.orderId, order.product, order.quantity, order.status, order.timestamp, order.emissions, order.sender);
    }

    function getTotalEmissions() public view returns (uint256) {
        return totalEmissions;
    }
}
"""

# Load and preprocess DataCo dataset
def load_data():
    try:
        file_path = '/content/DataCoSupplyChainDataset.csv'
        df = pd.read_csv(file_path, encoding='latin-1')
        print(f"Successfully loaded dataset with {df.shape[0]} rows and {df.shape[1]} columns")

        df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'], errors='coerce')
        if df['order date (DateOrders)'].isna().sum() > 0:
            print(f"Warning: {df['order date (DateOrders)'].isna().sum()} invalid dates set to NaT")

        numerical_cols = ['Order Item Quantity', 'Sales', 'Days for shipping (real)']
        for col in numerical_cols:
            df[col] = df[col].fillna(df[col].mean())
        df.fillna(0, inplace=True)
        print(f"Missing values handled: {numerical_cols} imputed with mean, others with 0")

        # Ensure Product Name is uniformly string
        df['Product Name'] = df['Product Name'].astype(str)
        # Validate Product Name entries
        invalid_entries = df['Product Name'].isna() | (df['Product Name'] == '')
        if invalid_entries.any():
            print(f"Warning: {invalid_entries.sum()} invalid Product Name entries replaced with 'Unknown'")
            df.loc[invalid_entries, 'Product Name'] = 'Unknown'

        le = LabelEncoder()
        df['product'] = le.fit_transform(df['Product Name'])
        print(f"Encoded {len(le.classes_)} unique products")

        required_cols = ['order date (DateOrders)', 'Order Item Quantity', 'Product Name', 'product', 'Sales', 'Days for shipping (real)', 'Shipping Mode']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")

        # Filter for high-demand products
        df = df[df['Order Item Quantity'] >= 5]
        print(f"Filtered dataset to {df.shape[0]} rows with Order Item Quantity >= 5")

        return df, le
    except FileNotFoundError:
        print(f"Error: {file_path} not found. Please upload DataCoSupplyChainDataset.csv to /content/.")
        print("Download from: https://data.mendeley.com/datasets/8gx2fvg2k6/5")
        print("Use: from google.colab import files; uploaded = files.upload()")
        return None, None
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return None, None

# Prepare data for LSTM
def prepare_lstm_data(df, product_id, scaler):
    product_data = df[df['product'] == product_id][['order date (DateOrders)', 'Order Item Quantity']].set_index('order date (DateOrders)')
    product_data = product_data.resample('D').sum().fillna(0)
    data = product_data['Order Item Quantity'].values
    data = scaler.fit_transform(data.reshape(-1, 1)).flatten()
    X, y = [], []
    for i in range(len(data) - 30):
        X.append(data[i:i+30])
        y.append(data[i+30])
    return np.array(X), np.array(y), scaler

# Train LSTM model
def train_lstm(df):
    product_id = df['product'].iloc[0]
    scaler = MinMaxScaler()
    X, y, scaler = prepare_lstm_data(df, product_id, scaler)
    if len(X) == 0:
        print("Error: Insufficient data for LSTM training")
        return None, None, None
    X = X.reshape((X.shape[0], X.shape[1], 1))
    inputs = Input(shape=(30, 1))
    lstm = LSTM(50, activation='relu')(inputs)
    outputs = Dense(1)(lstm)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=50, batch_size=32, verbose=0)
    y_pred = model.predict(X, verbose=0)
    mae = mean_absolute_error(scaler.inverse_transform(y.reshape(-1, 1)), scaler.inverse_transform(y_pred))
    rmse = np.sqrt(mean_squared_error(scaler.inverse_transform(y.reshape(-1, 1)), scaler.inverse_transform(y_pred)))
    print(f"LSTM MAE: {mae:.2f} units, RMSE: {rmse:.2f} units")
    return model, product_id, scaler

# Train Random Forest
def train_rf(df):
    X = df[['Order Item Quantity', 'Sales', 'Days for shipping (real)']]
    y = df['Order Item Quantity'] * 1.5
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    y_pred = model.predict(X)
    return model, X.columns

# Optimize emissions
def optimize_emissions(df, product_id):
    product_data = df[df['product'] == product_id][['Shipping Mode', 'Order Item Quantity']]
    emissions_rates = {'Standard Shipping': 0.2, 'Second Class': 0.25, 'First Class': 0.3, 'Same Day': 0.35}
    total_emissions = product_data.groupby('Shipping Mode')['Order Item Quantity'].sum().to_dict()
    for mode in total_emissions:
        total_emissions[mode] *= emissions_rates.get(mode, 0.2)
    recommended_mode = min(emissions_rates, key=lambda x: emissions_rates[x])
    return recommended_mode, total_emissions

# Setup mock blockchain with eth-tester
def setup_mock_blockchain():
    try:
        print("Setting up mock blockchain with eth-tester...")
        tester = EthereumTester(backend=PyEVMBackend())
        w3 = Web3(EthereumTesterProvider(tester))
        account = tester.get_accounts()[0]

        # Compile Solidity contract
        compiled_sol = compile_source(CONTRACT_SOURCE, output_values=['abi', 'bin'])
        contract_interface = compiled_sol['<stdin>:SupplyChain']
        abi = contract_interface['abi']
        bytecode = contract_interface['bin']

        # Deploy contract
        contract = w3.eth.contract(abi=abi, bytecode=bytecode)
        tx_hash = contract.constructor().transact({'from': account})
        tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
        contract_address = tx_receipt.contractAddress
        contract_instance = w3.eth.contract(address=contract_address, abi=abi)

        return w3, contract_instance, account
    except Exception as e:
        print(f"Error setting up mock blockchain: {str(e)}")
        return None, None, None

# Main execution
def main():
    print("=== Decentralized Autonomous Supply Chain Organization (DASCO) ===")
    print("Demonstrating ML and Ethereum Blockchain with Proof of Green Compliance\n")

    # Setup blockchain
    w3, contract, account = setup_mock_blockchain()
    if w3 is None:
        print("Failed to setup blockchain. Exiting...")
        return
    print(f"Mock Blockchain: Contract deployed at {contract.address}")

    # Load dataset
    df, le = load_data()
    if df is None:
        return

    # Train ML models
    lstm_model, product_id, scaler = train_lstm(df)
    rf_model, rf_columns = train_rf(df)

    if lstm_model is None or product_id is None:
        return

    # ML predictions
    X, _, _ = prepare_lstm_data(df, product_id, scaler)
    if len(X) == 0:
        print("Error: No data available for prediction")
        return
    X = X[-1].reshape((1, 30, 1))
    demand_scaled = lstm_model.predict(X, verbose=0)[0][0]
    demand = scaler.inverse_transform([[demand_scaled]])[0][0]

    sample = pd.DataFrame([df[['Order Item Quantity', 'Sales', 'Days for shipping (real)']].iloc[0].values], columns=rf_columns)
    optimal_stock = rf_model.predict(sample)[0]

    recommended_mode, emissions_data = optimize_emissions(df, product_id)

    # Blockchain transactions
    product_name = le.inverse_transform([product_id])[0]
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    current_quantity = 2  # Adjusted to trigger reorder
    threshold = int(optimal_stock)

    try:
        # Create order
        order_id = 'TX001'
        quantity = current_quantity
        emissions = int(quantity * 0.2 * 100)  # kg CO2 * 100
        tx_hash = contract.functions.createOrder(order_id, product_name, quantity, emissions).transact({'from': account, 'gas': 1000000})
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Blockchain: Created order {order_id} for {quantity} units of {product_name} (Emissions: {emissions/100:.2f} kg CO2, Tx Hash: {tx_hash.hex()})")

        # Update shipment
        emissions_update = int(quantity * 0.1 * 100)
        tx_hash = contract.functions.updateShipment(order_id, "In Transit", emissions_update).transact({'from': account, 'gas': 1000000})
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Blockchain: Updated order {order_id} to 'In Transit' (Emissions: {emissions_update/100:.2f} kg CO2, Tx Hash: {tx_hash.hex()})")

        # Automate reorder
        new_order_id = f"REORDER_TX001_{int(time.time())}"
        emissions_reorder = int((threshold - current_quantity) * 0.1 * 100)
        tx_hash = contract.functions.automateReorder(order_id, product_name, current_quantity, threshold, new_order_id, emissions_reorder).transact({'from': account, 'gas': 1000000})
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Blockchain: Triggered reorder {new_order_id} for {threshold - current_quantity} units (Emissions: {emissions_reorder/100:.2f} kg CO2, Tx Hash: {tx_hash.hex()})")

        # Retrieve transactions
        order_count = contract.functions.orderCount().call()
        total_emissions = contract.functions.getTotalEmissions().call() / 100
        orders = []
        for order_id_to_fetch in ['TX001', new_order_id]:
            try:
                order_data = contract.functions.getOrder(order_id_to_fetch).call()
                orders.append({
                    'orderId': order_data[0],
                    'product': order_data[1],
                    'quantity': order_data[2],
                    'status': order_data[3],
                    'timestamp': datetime.fromtimestamp(order_data[4]).strftime('%Y-%m-%d %H:%M:%S'),
                    'emissions': order_data[5] / 100,
                    'sender': order_data[6]
                })
            except:
                continue

    except Exception as e:
        print(f"Blockchain Error: {str(e)}")
        return

    # Display results
    print("\n1. ML Demand Prediction:")
    print(f"   Product: {product_name}")
    print(f"   Predicted Demand: {round(float(demand), 2)} units")
    print(f"   Date: {timestamp.split()[0]}\n")

    print("2. Inventory Optimization:")
    print(f"   Product: {product_name}")
    print(f"   Optimal Stock: {round(float(optimal_stock), 2)} units")
    print(f"   Current Stock: {current_quantity} units")
    print(f"   Reorder Threshold: {threshold} units\n")

    print("3. Sustainability Analysis:")
    print(f"   Recommended Shipping Mode: {recommended_mode} (lowest emissions)")
    print(f"   Current Emissions by Mode: {emissions_data}")
    print(f"   Total Emissions Recorded on Blockchain: {total_emissions:.2f} kg CO2\n")

    print("4. Blockchain Transactions (Proof of Green Compliance):")
    for order in orders:
        print(f"   ID: {order['orderId']}, Product: {order['product']}, Quantity: {order['quantity']}, "
              f"Status: {order['status']}, Emissions: {order['emissions']:.2f} kg CO2, Timestamp: {order['timestamp']}, Sender: {order['sender']}")

    print("\n=== Summary ===")
    print("This prototype demonstrates DASCO's integration of ML and Ethereum blockchain with Proof of Green Compliance, ensuring transparent, automated, and sustainable supply chain management.")

if __name__ == '__main__':
    main()

=== Decentralized Autonomous Supply Chain Organization (DASCO) ===
Demonstrating ML and Ethereum Blockchain with Proof of Green Compliance

Setting up mock blockchain with eth-tester...
Mock Blockchain: Contract deployed at 0xF2E246BB76DF876Cef8b38ae84130F4F55De395b
Successfully loaded dataset with 47560 rows and 53 columns
Missing values handled: ['Order Item Quantity', 'Sales', 'Days for shipping (real)'] imputed with mean, others with 0
Encoded 98 unique products
Filtered dataset to 6959 rows with Order Item Quantity >= 5
LSTM MAE: 5.36 units, RMSE: 6.72 units
Blockchain: Created order TX001 for 2 units of Nike Men's Dri-FIT Victory Golf Polo (Emissions: 0.40 kg CO2, Tx Hash: 8c94a2c4a8f975e2c2124a3de7657ae31b73963b9d11b2f09a6a6ca33a0027c4)
Blockchain: Updated order TX001 to 'In Transit' (Emissions: 0.20 kg CO2, Tx Hash: 327d1e6d4a38a4f952f2414974f149526308fd40407483d38e6878b4a154661b)
Blockchain: Triggered reorder REORDER_TX001_1763885686 for 5 units (Emissions: 0.50 kg CO2, Tx Has

In [ ]:
# DASCO prototype — improved single-file script\n# Features:\n# - Robust data loading and validation\n# - LSTM forecasting with train/validation, scaler handling\n# - RandomForest predicting order quantity (train/test + scaler)\n# - Correct emissions optimization (min total emissions)\n# - Robust eth-tester mock blockchain setup & solidity compile/deploy\n# - Clear prints and error handling\n\nimport os\nimport time\nfrom datetime import datetime\n\nimport numpy as np\nimport pandas as pd\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler\n\nfrom tensorflow.keras.models import Model\nfrom tensorflow.keras.layers import Input, LSTM, Dense\nfrom tensorflow.keras.callbacks import EarlyStopping\n\n# blockchain imports\nfrom web3 import Web3\nfrom solcx import compile_source, install_solc\nfrom eth_tester import EthereumTester, PyEVMBackend\nfrom web3.providers.eth_tester import EthereumTesterProvider\n\nfrom sklearn.ensemble import IsolationForest\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n\n# Solidity contract (unchanged structure except comments)\nCONTRACT_SOURCE = """\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract SupplyChain {\n    struct Order {\n        string orderId;\n        string product;\n        uint256 quantity;\n        string status;\n        uint256 timestamp;\n        uint256 emissions; // in kg CO2 * 100 for precision\n        address sender;\n    }\n\n    mapping(string => Order) public orders;\n    uint256 public orderCount;\n    uint256 public totalEmissions; // in kg CO2 * 100\n    uint256 public emissionThreshold = 1000; // 10 kg CO2 * 100\n\n    event OrderCreated(string orderId, string product, uint256 quantity, uint256 emissions);\n    event ShipmentUpdated(string orderId, string status, uint256 emissions);\n    event ReorderTriggered(string orderId, string product, uint256 quantity);\n\n    function createOrder(string memory orderId, string memory product, uint256 quantity, uint256 emissions) public {\n        require(emissions <= emissionThreshold, "Emissions exceed Proof of Green Compliance threshold");\n        require(bytes(orders[orderId].orderId).length == 0, "Order ID already exists");\n\n        orders[orderId] = Order(orderId, product, quantity, "Created", block.timestamp, emissions, msg.sender);\n        orderCount++;\n        totalEmissions += emissions;\n        emit OrderCreated(orderId, product, quantity, emissions);\n    }\n\n    function updateShipment(string memory orderId, string memory status, uint256 emissions) public {\n        require(bytes(orders[orderId].orderId).length != 0, "Order does not exist");\n        require(emissions <= emissionThreshold, "Emissions exceed Proof of Green Compliance threshold");\n\n        orders[orderId].status = status;\n        orders[orderId].emissions += emissions;\n        totalEmissions += emissions;\n        emit ShipmentUpdated(orderId, status, emissions);\n    }\n\n    function automateReorder(string memory orderId, string memory product, uint256 currentQuantity, uint256 threshold, string memory newOrderId, uint256 emissions) public {\n        require(currentQuantity < threshold, "Current quantity above threshold");\n        createOrder(newOrderId, product, threshold - currentQuantity, emissions);\n        emit ReorderTriggered(newOrderId, product, threshold - currentQuantity);\n    }\n\n    function getOrder(string memory orderId) public view returns (string memory, string memory, uint256, string memory, uint256, uint256, address) {\n        Order memory order = orders[orderId];\n        return (order.orderId, order.product, order.quantity, order.status, order.timestamp, order.emissions, order.sender);\n    }\n\n    function getTotalEmissions() public view returns (uint256) {\n        return totalEmissions;\n    }\n}\n"""\n\n# -------------------------\n# Data loading & preprocessing\n# -------------------------\ndef load_data(file_path='/content/DataCoSupplyChainDataset.csv', min_qty_filter=None):\n    """\n    Load the DataCo dataset with safer checks.\n    min_qty_filter: if provided, filter Order Item Quantity >= min_qty_filter (optional).\n    Returns: df (pandas.DataFrame), label_encoder (LabelEncoder for product names)\n    """\n    if not os.path.exists(file_path):\n        print(f"Error: dataset not found at {file_path}. Please upload the file or change file_path.")\n        return None, None\n\n    try:\n        df = pd.read_csv(file_path, encoding='latin-1')\n        # normalize column names (strip whitespace)\n        df.columns = [c.strip() for c in df.columns]\n\n        # required columns (adapt if your CSV has slightly different names)\n        required_cols = ['order date (DateOrders)', 'Order Item Quantity', 'Product Name', 'Sales', 'Days for shipping (real)', 'Shipping Mode']\n        missing_cols = [c for c in required_cols if c not in df.columns]\n        if missing_cols:\n            raise ValueError(f"Missing required columns: {missing_cols}")\n\n        # parse dates\n        df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'], errors='coerce')\n        if df['order date (DateOrders)'].isna().sum() > 0:\n            print(f"Warning: {df['order date (DateOrders)'].isna().sum()} invalid dates set to NaT")\n\n        # numeric conversions & imputation for numeric columns only\n        num_cols = ['Order Item Quantity', 'Sales', 'Days for shipping (real)']\n        for col in num_cols:\n            df[col] = pd.to_numeric(df[col], errors='coerce')\n            mean_val = df[col].mean()\n            df[col].fillna(mean_val, inplace=True)\n\n        # Product Name handling\n        df['Product Name'] = df['Product Name'].astype(str).fillna('Unknown')\n        invalid_entries = df['Product Name'].isna() | (df['Product Name'] == '')\n        if invalid_entries.any():\n            df.loc[invalid_entries, 'Product Name'] = 'Unknown'\n\n        # encode product\n        le = LabelEncoder()\n        df['product'] = le.fit_transform(df['Product Name'])\n\n        # optional filter by minimum order item quantity\n        if min_qty_filter is not None:\n            before = len(df)\n            df = df[df['Order Item Quantity'] >= min_qty_filter].copy()\n            after = len(df)\n            print(f"Applied min_qty_filter={min_qty_filter}: rows {before} -> {after}")\n\n        print(f"Loaded dataset with shape: {df.shape} and {len(le.classes_)} unique products")\n        return df, le\n\n    except Exception as e:\n        print("Error loading data:", e)\n        return None, None\n\n# -------------------------\n# LSTM time-series preparation and training\n# -------------------------\ndef prepare_lstm_data(df, product_id, scaler=None, lookback=30):\n    """\n    Prepare daily-resampled time series for a particular product.\n    Returns X (num_samples, lookback), y (num_samples,), scaler (fitted)\n    """\n    prod_df = df[df['product'] == product_id][['order date (DateOrders)', 'Order Item Quantity']].copy()\n    if prod_df.empty:\n        return np.array([]), np.array([]), scaler\n\n    prod_df = prod_df.set_index('order date (DateOrders)').sort_index()\n    # resample to daily frequency and sum quantities per day\n    prod_daily = prod_df.resample('D').sum().fillna(0)\n    data = prod_daily['Order Item Quantity'].values.reshape(-1, 1)\n\n    if scaler is None:\n        scaler = MinMaxScaler()\n    data_scaled = scaler.fit_transform(data)\n\n    X, y = [], []\n    for i in range(len(data_scaled) - lookback):\n        X.append(data_scaled[i:i + lookback, 0])\n        y.append(data_scaled[i + lookback, 0])\n\n    return np.array(X), np.array(y), scaler\n\ndef train_lstm(df, product_id=None, lookback=30, epochs=30):\n    """\n    Train an LSTM on the product time series.\n    If product_id is None, chooses product with most records.\n    Returns model, product_id, scaler\n    """\n    if product_id is None:\n        product_id = df['product'].value_counts().idxmax()\n        print(f"No product_id provided. Using most frequent product: {product_id}")\n\n    scaler = MinMaxScaler()\n    X, y, scaler = prepare_lstm_data(df, product_id, scaler=scaler, lookback=lookback)\n    if len(X) == 0:\n        print("Insufficient data for LSTM training")\n        return None, None, None\n\n    # train/validation split (time-order preserved)\n    split = int(len(X) * 0.8)\n    X_train, X_val = X[:split], X[split:]\n    y_train, y_val = y[:split], y[split:]\n\n    # reshape for LSTM\n    X_train = X_train.reshape((X_train.shape[0], lookback, 1))\n    X_val = X_val.reshape((X_val.shape[0], lookback, 1))\n\n    inputs = Input(shape=(lookback, 1))\n    lstm = LSTM(50)(inputs)  # default tanh internal activations\n    outputs = Dense(1)(lstm)\n    model = Model(inputs=inputs, outputs=outputs)\n    model.compile(optimizer='adam', loss='mse')\n\n    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)\n    print(f"Training LSTM for product {product_id} — epochs={epochs}, lookback={lookback}")\n    model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_val, y_val), callbacks=[es], verbose=1)\n\n    # validation metrics\n    if len(X_val) > 0:\n        y_val_pred = model.predict(X_val)\n        y_val_inv = scaler.inverse_transform(y_val.reshape(-1, 1)).flatten()\n        y_val_pred_inv = scaler.inverse_transform(y_val_pred).flatten()\n        val_mae = mean_absolute_error(y_val_inv, y_val_pred_inv)\n        val_rmse = np.sqrt(mean_squared_error(y_val_inv, y_val_pred_inv))\n        print(f"LSTM Validation MAE: {val_mae:.2f}, RMSE: {val_rmse:.2f}")\n    else:\n        print("No validation split available to report metrics.")\n\n    return model, product_id, scaler\n\n# -------------------------\n# Random Forest training (predicting Order Item Quantity)\n# -------------------------\ndef train_rf(df):\n    """\n    Train RF to predict Order Item Quantity using Sales and Days for shipping (real).\n    Returns model, feature_columns (list), scaler\n    """\n    X = df[['Sales', 'Days for shipping (real)']].copy()\n    y = df['Order Item Quantity'].copy().astype(float)\n\n    # simple train/test split\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n    scaler = StandardScaler()\n    X_train_sc = scaler.fit_transform(X_train)\n    X_test_sc = scaler.transform(X_test)\n\n    model = RandomForestRegressor(n_estimators=100, random_state=42)\n    model.fit(X_train_sc, y_train)\n\n    y_pred = model.predict(X_test_sc)\n    print(f"RF Test MAE: {mean_absolute_error(y_test, y_pred):.2f}, RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}, R2: {r2_score(y_test, y_pred):.3f}")\n\n    return model, X.columns.tolist(), scaler\n\n# -------------------------\n# Emissions optimization\n# -------------------------\ndef optimize_emissions(df, product_id):\n    """\n    Compute total emissions per shipping mode for a product and recommend the mode that\n    minimizes total emissions (qty * per-unit-rate).\n    Returns recommended_mode, dict(total_emissions_by_mode)\n    """\n    product_data = df[df['product'] == product_id][['Shipping Mode', 'Order Item Quantity']].copy()\n    # default per-unit emissions (kg CO2 per unit)\n    emissions_rates = {'Standard Shipping': 0.2, 'Second Class': 0.25, 'First Class': 0.3, 'Same Day': 0.35}\n\n    totals = product_data.groupby('Shipping Mode')['Order Item Quantity'].sum().to_dict()\n    total_emissions_by_mode = {mode: totals.get(mode, 0.0) * emissions_rates.get(mode, 0.2) for mode in emissions_rates}\n\n    recommended_mode = min(total_emissions_by_mode, key=lambda m: total_emissions_by_mode[m])\n    return recommended_mode, total_emissions_by_mode\n\n# -------------------------\n# Mock blockchain setup (eth-tester) with robust deploy/tx handling\n# -------------------------\ndef setup_mock_blockchain():\n    """\n    Sets up an eth-tester blockchain, compiles the solidity contract, deploys and returns:\n    w3 (Web3), contract_instance (Contract), account (deployer address)\n    """\n    try:\n        print("Attempting to install solc 0.8.0 (if not already installed)...")\n        try:\n            install_solc('0.8.0')\n        except Exception as e:\n            print("Warning: install_solc failed or network not available. If compile fails, install a local solc 0.8.x manually. Error:", e)\n\n        print("Setting up eth-tester backend...")\n        tester = EthereumTester(backend=PyEVMBackend())\n        w3 = Web3(EthereumTesterProvider(tester))\n        account = tester.get_accounts()[0]\n        w3.eth.default_account = account\n\n        # compile\n        compiled = compile_source(CONTRACT_SOURCE, output_values=['abi', 'bin'])\n        contract_key = next(iter(compiled.keys()))\n        contract_interface = compiled[contract_key]\n        abi = contract_interface['abi']\n        bytecode = contract_interface['bin']\n\n        # deploy (wait for receipt)\n        contract_factory = w3.eth.contract(abi=abi, bytecode=bytecode)\n        tx_hash = contract_factory.constructor().transact()\n        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)\n        contract_address = tx_receipt.contractAddress\n        contract_instance = w3.eth.contract(address=contract_address, abi=abi)\n\n        print(f"Deployed contract at address: {contract_address}")\n        return w3, contract_instance, account\n\n    except Exception as e:\n        print("Error setting up mock blockchain:", e)\n        return None, None, None\n\n# -------------------------\n# Main execution\n# -------------------------\n\nclass AnomalyDetector:\n    """NEW FEATURE 1: Real-time Anomaly Detection using Isolation Forest"""\n    def __init__(self, contamination=0.1):\n        self.model = IsolationForest(contamination=contamination, random_state=42)\n        self.scaler = StandardScaler()\n\n    def train(self, df):\n        features = df[['Order Item Quantity', 'Sales', 'Days for shipping (real)']].copy()\n        features_scaled = self.scaler.fit_transform(features)\n        self.model.fit(features_scaled)\n        print("✓ Anomaly Detector trained")\n\n    def detect(self, df):\n        features = df[['Order Item Quantity', 'Sales', 'Days for shipping (real)']].copy()\n        features_scaled = self.scaler.transform(features)\n        predictions = self.model.predict(features_scaled)\n        anomaly_scores = self.model.score_samples(features_scaled)\n\n        df['is_anomaly'] = predictions == -1\n        df['anomaly_score'] = anomaly_scores\n\n        anomaly_count = df['is_anomaly'].sum()\n        print(f"✓ Detected {anomaly_count} anomalies ({anomaly_count/len(df)*100:.2f}%)")\n        return df\n\n\nclass PortfolioOptimizer:\n    """NEW FEATURE 2: Multi-Product Portfolio Optimization"""\n    def __init__(self, df, le):\n        self.df = df\n        self.le = le\n\n    def optimize_portfolio(self, top_n=5):\n        print(f"\n🔍 Analyzing top {top_n} products for portfolio optimization...")\n\n        # Calculate metrics per product\n        product_stats = self.df.groupby('product').agg({\n            'Order Item Quantity': ['sum', 'mean', 'std'],\n            'Sales': 'sum',\n            'Days for shipping (real)': 'mean'\n        }).reset_index()\n\n        product_stats.columns = ['product', 'total_qty', 'avg_qty', 'std_qty', 'total_sales', 'avg_shipping_days']\n        product_stats['revenue_per_unit'] = product_stats['total_sales'] / product_stats['total_qty']\n        product_stats['consistency_score'] = 1 / (1 + product_stats['std_qty'])\n        product_stats['efficiency_score'] = 1 / (1 + product_stats['avg_shipping_days'])\n\n        # Composite score\n        product_stats['portfolio_score'] = (\n            0.4 * product_stats['revenue_per_unit'] / product_stats['revenue_per_unit'].max() +\n            0.3 * product_stats['consistency_score'] +\n            0.3 * product_stats['efficiency_score']\n        )\n\n        top_products = product_stats.nlargest(top_n, 'portfolio_score')\n\n        results = []\n        for _, row in top_products.iterrows():\n            product_name = self.le.inverse_transform([int(row['product'])])[0]\n            results.append({\n                'product_id': int(row['product']),\n                'product_name': product_name,\n                'portfolio_score': row['portfolio_score'],\n                'total_sales': row['total_sales'],\n                'avg_qty': row['avg_qty']\n            })\n\n        print(f"✓ Portfolio optimized with {len(results)} products")\n        return results\n\n\nclass DemandCorrelationAnalyzer:\n    """Feature 3: Cross-Product Demand Correlation Analysis"""\n    def __init__(self, df):\n        self.df = df\n\n    def analyze_correlations(self, top_n=10):\n        print(f"  Analyzing correlations for top {top_n} products...")\n\n        top_products = self.df['product'].value_counts().head(top_n).index\n\n        daily_demand = self.df[self.df['product'].isin(top_products)].pivot_table(\n            index='order date (DateOrders)',\n            columns='product',\n            values='Order Item Quantity',\n            aggfunc='sum'\n        ).fillna(0)\n\n        correlation_matrix = daily_demand.corr()\n\n        correlations = []\n        for i in range(len(correlation_matrix)):\n            for j in range(i+1, len(correlation_matrix)):\n                correlations.append({\n                    'product1': correlation_matrix.index[i],\n                    'product2': correlation_matrix.columns[j],\n                    'correlation': correlation_matrix.iloc[i, j]\n                })\n\n        correlations_df = pd.DataFrame(correlations)\n        correlations_df = correlations_df.sort_values('correlation', ascending=False)\n\n        print(f"  ✓ Found {len(correlations_df)} product correlations")\n        return correlations_df.head(5), correlation_matrix\n\n\n\nclass VisualizationDashboard:\n    """Feature 6: Advanced Visualization Dashboard"""\n    @staticmethod\n    def create_dashboard(df, portfolio_results, anomaly_df):\n        print("  Creating dashboard...")\n\n        fig = plt.figure(figsize=(16, 10))\n        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)\n\n        # 1. Order Quantity Distribution\n        ax1 = fig.add_subplot(gs[0, 0])\n        ax1.hist(df['Order Item Quantity'], bins=50, color='skyblue', edgecolor='black')\n        ax1.set_title('Order Quantity Distribution', fontweight='bold')\n        ax1.set_xlabel('Quantity')\n        ax1.set_ylabel('Frequency')\n\n        # 2. Top Products by Sales\n        ax2 = fig.add_subplot(gs[0, 1])\n        top_sales = df.groupby('Product Name')['Sales'].sum().nlargest(8)\n        ax2.barh(range(len(top_sales)), top_sales.values, color='coral')\n        ax2.set_yticks(range(len(top_sales)))\n        ax2.set_yticklabels([name[:25] + '...' if len(name) > 25 else name for name in top_sales.index], fontsize=8)\n        ax2.set_title('Top 8 Products by Sales', fontweight='bold')\n        ax2.set_xlabel('Total Sales ($)')\n\n        # 3. Shipping Mode Pie Chart\n        ax3 = fig.add_subplot(gs[0, 2])\n        shipping_dist = df['Shipping Mode'].value_counts()\n        colors = plt.cm.Set3(range(len(shipping_dist)))\n        ax3.pie(shipping_dist.values, labels=shipping_dist.index, autopct='%1.1f%%',\n               startangle=90, colors=colors)\n        ax3.set_title('Shipping Mode Distribution', fontweight='bold')\n\n        # 4. Anomaly Detection Scatter\n        ax4 = fig.add_subplot(gs[1, 0])\n        if 'is_anomaly' in anomaly_df.columns:\n            normal = anomaly_df[anomaly_df['is_anomaly'] == False]\n            anomaly = anomaly_df[anomaly_df['is_anomaly'] == True]\n            ax4.scatter(normal['Sales'], normal['Order Item Quantity'],\n                       alpha=0.3, s=5, label='Normal', color='blue')\n            ax4.scatter(anomaly['Sales'], anomaly['Order Item Quantity'],\n                       alpha=0.8, s=30, label='Anomaly', color='red', marker='x')\n            ax4.set_title('Anomaly Detection Results', fontweight='bold')\n            ax4.set_xlabel('Sales ($)')\n            ax4.set_ylabel('Order Quantity')\n            ax4.legend()\n\n        # 5. Portfolio Scores\n        ax5 = fig.add_subplot(gs[1, 1])\n        if portfolio_results:\n            portfolio_df = pd.DataFrame(portfolio_results)\n            colors_portfolio = plt.cm.Greens(np.linspace(0.4, 0.8, len(portfolio_df)))\n            ax5.barh(range(len(portfolio_df)), portfolio_df['portfolio_score'], color=colors_portfolio)\n            ax5.set_yticks(range(len(portfolio_df)))\n            ax5.set_yticklabels([name[:20] + '...' if len(name) > 20 else name\n                                for name in portfolio_df['product_name']], fontsize=8)\n            ax5.set_title('Portfolio Optimization Scores', fontweight='bold')\n            ax5.set_xlabel('Score')\n\n        # 6. Emissions by Mode\n        ax6 = fig.add_subplot(gs[1, 2])\n        emissions_rates = {'Standard Shipping': 0.2, 'Second Class': 0.25,\n                          'First Class': 0.3, 'Same Day': 0.35}\n        emissions_by_mode = {}\n        for mode, rate in emissions_rates.items():\n            qty = df[df['Shipping Mode'] == mode]['Order Item Quantity'].sum()\n            emissions_by_mode[mode] = qty * rate\n\n        ax6.bar(range(len(emissions_by_mode)), list(emissions_by_mode.values()),\n               color='orange', edgecolor='black')\n        ax6.set_xticks(range(len(emissions_by_mode)))\n        ax6.set_xticklabels([m.replace(' ', '\n') for m in emissions_by_mode.keys()],\n                            fontsize=8, rotation=0)\n        ax6.set_title('Emissions by Shipping Mode', fontweight='bold')\n        ax6.set_ylabel('Emissions (kg CO2)')\n\n        # 7. Sales Trend Over Time\n        ax7 = fig.add_subplot(gs[2, :2])\n        monthly_sales = df.groupby(df['order date (DateOrders)'].dt.to_period('M'))['Sales'].sum()\n        ax7.plot(range(len(monthly_sales)), monthly_sales.values,\n                marker='o', linewidth=2, color='darkgreen')\n        ax7.set_title('Monthly Sales Trend', fontweight='bold')\n        ax7.set_xlabel('Month Index')\n        ax7.set_ylabel('Total Sales ($)')\n        ax7.grid(True, alpha=0.3)\n\n        # 8. Demand Heatmap (simplified)\n        ax8 = fig.add_subplot(gs[2, 2])\n        top_5_products = df['product'].value_counts().head(5).index\n        heatmap_data = df[df['product'].isin(top_5_products)].groupby(\n            ['product', df['order date (DateOrders)'].dt.dayofweek]\n        )['Order Item Quantity'].sum().unstack(fill_value=0)\n\n        if not heatmap_data.empty:\n            sns.heatmap(heatmap_data, cmap='YlOrRd', ax=ax8, cbar_kws={'label': 'Quantity'})\n            ax8.set_title('Demand by Day of Week', fontweight='bold')\n            ax8.set_xlabel('Day of Week')\n            ax8.set_ylabel('Product ID')\n\n        plt.suptitle('DASCO Enhanced Supply Chain Analytics Dashboard',\n                    fontsize=18, fontweight='bold', y=0.995)\n\n        plt.savefig('dasco_dashboard.png', dpi=300, bbox_inches='tight')\n        print("  ✓ Dashboard saved: dasco_dashboard.png")\n        plt.show()\n\n\ndef main():\n    print("=== Decentralized Autonomous Supply Chain Organization (DASCO) ===\n")\n\n    # 1) Setup blockchain\n    w3, contract, account = setup_mock_blockchain()\n    if w3 is None:\n        print("Failed to setup blockchain. Exiting...")\n        return\n    print(f"Mock Blockchain ready. Deployer account: {account}\n")\n\n    # 2) Load dataset\n    df, le = load_data('/content/DataCoSupplyChainDataset.csv', min_qty_filter=None)\n    if df is None:\n        print("Dataset not loaded. Exiting.")\n        return\n\n    # --- NEW: Anomaly Detection ---\n    print("\n--- Feature 1: Anomaly Detection ---")\n    ad = AnomalyDetector()\n    ad.train(df)\n    df = ad.detect(df)\n\n    # 3) Train ML models\n    lstm_model, lstm_product_id, lstm_scaler = train_lstm(df, product_id=None, lookback=30, epochs=20)\n    rf_model, rf_columns, rf_scaler = train_rf(df)\n\n    if lstm_model is None or lstm_product_id is None:\n        print("LSTM model not trained due to insufficient data. Continuing with RF & blockchain demo.\n")\n\n    # 4) Make predictions / compute optimal stock\n    predicted_demand = None\n    if lstm_model is not None:\n        X_all, _, _ = prepare_lstm_data(df, lstm_product_id, scaler=lstm_scaler, lookback=30)\n        if len(X_all) > 0:\n            last_window = X_all[-1].reshape((1, 30, 1))\n            demand_scaled = float(lstm_model.predict(last_window, verbose=0)[0][0])\n            predicted_demand = float(lstm_scaler.inverse_transform([[demand_scaled]])[0][0])\n        else:\n            print("No LSTM windows available for prediction.")\n\n    # For RF: create a sample from the first row of df (features must match rf_columns)\n    sample_vals = df[rf_columns].iloc[0].values.reshape(1, -1)\n    sample_scaled = rf_scaler.transform(sample_vals)\n    optimal_stock = float(rf_model.predict(sample_scaled)[0])\n\n    # --- NEW: Portfolio Optimization ---\n    print("\n--- Feature 2: Portfolio Optimization ---")\n    po = PortfolioOptimizer(df, le)\n    portfolio_results = po.optimize_portfolio(top_n=5)\n\n    # --- NEW: Demand Correlation ---\n    print("\n--- Feature 3: Demand Correlation ---")\n    dca = DemandCorrelationAnalyzer(df)\n    correlations, _ = dca.analyze_correlations(top_n=5)\n\n    # 5) Emissions recommendation\n    product_for_emissions = lstm_product_id if lstm_product_id is not None else df['product'].value_counts().idxmax()\n    recommended_mode, emissions_by_mode = optimize_emissions(df, product_for_emissions)\n\n    # 6) Blockchain demo\n    product_name = le.inverse_transform([int(product_for_emissions)])[0]\n    timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')\n    current_quantity = 2\n    threshold = max(1, int(round(optimal_stock)))\n\n    orders = []\n    try:\n        order_id = "TX001"\n        quantity = current_quantity\n        emissions = int(quantity * 0.2 * 100)\n\n        tx_hash = contract.functions.createOrder(order_id, product_name, quantity, emissions).transact({'from': account})\n        w3.eth.wait_for_transaction_receipt(tx_hash)\n        print(f"Blockchain: Created order {order_id}")\n\n        emissions_update = int(quantity * 0.1 * 100)\n        tx_hash = contract.functions.updateShipment(order_id, "In Transit", emissions_update).transact({'from': account})\n        w3.eth.wait_for_transaction_receipt(tx_hash)\n        print(f"Blockchain: Updated shipment for {order_id}")\n\n        if current_quantity < threshold:\n            new_order_id = f"REORDER_{int(time.time())}"\n            emissions_reorder = int((threshold - current_quantity) * 0.1 * 100)\n            tx_hash = contract.functions.automateReorder(order_id, product_name, current_quantity, threshold, new_order_id, emissions_reorder).transact({'from': account})\n            w3.eth.wait_for_transaction_receipt(tx_hash)\n            print(f"Blockchain: Triggered reorder {new_order_id}")\n        else:\n            new_order_id = None\n\n        fetched_ids = [order_id]\n        if new_order_id:\n            fetched_ids.append(new_order_id)\n\n        for oid in fetched_ids:\n            try:\n                od = contract.functions.getOrder(oid).call()\n                ts = od[4]\n                ts_str = datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S') if ts != 0 else "0"\n                orders.append({\n                    'orderId': od[0],\n                    'product': od[1],\n                    'quantity': od[2],\n                    'status': od[3],\n                    'timestamp': ts_str,\n                    'emissions': od[5] / 100.0,\n                    'sender': od[6]\n                })\n            except Exception:\n                continue\n\n        total_emissions = contract.functions.getTotalEmissions().call() / 100.0\n\n    except Exception as e:\n        print("Blockchain Error:", e)\n        return\n\n    # Display Results\n    print("\n=== Results ===\n")\n    print("1) ML Demand Prediction:")\n    if predicted_demand is not None:\n        print(f"   Product: {product_name}")\n        print(f"   Predicted Demand: {predicted_demand:.2f} units")\n    else:\n        print(f"   LSTM prediction unavailable.")\n    \n    print("\n2) Inventory Optimization:")\n    print(f"   Optimal Stock: {optimal_stock:.2f} units")\n    \n    print("\n3) Sustainability Analysis:")\n    print(f"   Recommended Mode: {recommended_mode}")\n    print(f"   Total Emissions (Blockchain): {total_emissions:.2f} kg CO2")\n\n    print("\n4) Blockchain Transactions:")\n    for o in orders:\n        print(f"   ID: {o['orderId']}, Status: {o['status']}, Emissions: {o['emissions']:.2f}")\n\n    # --- NEW: Dashboard ---\n    print("\n--- Feature 6: Dashboard ---")\n    VisualizationDashboard.create_dashboard(df, portfolio_results, df)\n\n    print("\n=== Summary ===")\n    print("Integrated System: Blockchain + ML + Analytics + Dashboard")\n\nif __name__ == "__main__":\n    main()

=== Decentralized Autonomous Supply Chain Organization (DASCO) ===

Attempting to install solc 0.8.0 (if not already installed)...
Setting up eth-tester backend...
Deployed contract at address: 0xF2E246BB76DF876Cef8b38ae84130F4F55De395b
Mock Blockchain ready. Deployer account: 0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf



/tmp/ipython-input-499603115.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mean_val, inplace=True)


Loaded dataset with shape: (180519, 54) and 118 unique products
No product_id provided. Using most frequent product: 71
Training LSTM for product 71 — epochs=20, lookback=30
Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.1302 - val_loss: 0.0262
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0317 - val_loss: 0.0285
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0290 - val_loss: 0.0261
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0298 - val_loss: 0.0264
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0272 - val_loss: 0.0269
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0288 - val_loss: 0.0258
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0280 - val_loss: 0.0260
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0290 - val_loss: 0.0259
Epoch 9/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0296 - val_loss: 0.0257
Epoch 10/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0274 - v

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Blockchain: Created order TX001 (tx: 75eebc001c50e970f1282d43f9a3761f4d40008720cacdeeaf08ab5b6b1b5188)
Blockchain: Updated shipment for TX001 (tx: 1120f470058d541af3454587bdaad824026a8b4c4f45102e60bc2351558e0ae9)

=== Results ===

1) ML Demand Prediction:
   Product: Perfect Fitness Perfect Rip Deck
   Predicted Demand (next day / next window): 68.80 units
   Calculation Date: 2025-11-05 20:02:38

2) Inventory Optimization (via Random Forest):
   Product (sample row): Perfect Fitness Perfect Rip Deck
   RF-predicted optimal stock (approx): 1.00 units
   Current Stock: 2 units
   Reorder Threshold (int): 1 units

3) Sustainability Analysis:
   Recommended Shipping Mode (min total emissions): Standard Shipping
   Emissions by Mode (kg CO2): {'Standard Shipping': 0.0, 'Second Class': 3583.25, 'First Class': 3351.0, 'Same Day': 1461.25}
   Total Emissions Recorded on Blockchain: 0.60 kg CO2

4) Blockchain Transactions (Proof of Green Compliance):
   ID: TX001, Product: Perfect Fitness Perf

In [6]:
class AnomalyDetector:
    """NEW FEATURE 1: Real-time Anomaly Detection using Isolation Forest"""
    def __init__(self, contamination=0.1):
        self.model = IsolationForest(contamination=contamination, random_state=42)
        self.scaler = StandardScaler()

    def train(self, df):
        features = df[['Order Item Quantity', 'Sales', 'Days for shipping (real)']].copy()
        features_scaled = self.scaler.fit_transform(features)
        self.model.fit(features_scaled)
        print("✓ Anomaly Detector trained")

    def detect(self, df):
        features = df[['Order Item Quantity', 'Sales', 'Days for shipping (real)']].copy()
        features_scaled = self.scaler.transform(features)
        predictions = self.model.predict(features_scaled)
        anomaly_scores = self.model.score_samples(features_scaled)

        df['is_anomaly'] = predictions == -1
        df['anomaly_score'] = anomaly_scores

        anomaly_count = df['is_anomaly'].sum()
        print(f"✓ Detected {anomaly_count} anomalies ({anomaly_count/len(df)*100:.2f}%)")
        return df


In [3]:
class PortfolioOptimizer:
    """NEW FEATURE 2: Multi-Product Portfolio Optimization"""
    def __init__(self, df, le):
        self.df = df
        self.le = le

    def optimize_portfolio(self, top_n=5):
        print(f"\n🔍 Analyzing top {top_n} products for portfolio optimization...")

        # Calculate metrics per product
        product_stats = self.df.groupby('product').agg({
            'Order Item Quantity': ['sum', 'mean', 'std'],
            'Sales': 'sum',
            'Days for shipping (real)': 'mean'
        }).reset_index()

        product_stats.columns = ['product', 'total_qty', 'avg_qty', 'std_qty', 'total_sales', 'avg_shipping_days']
        product_stats['revenue_per_unit'] = product_stats['total_sales'] / product_stats['total_qty']
        product_stats['consistency_score'] = 1 / (1 + product_stats['std_qty'])
        product_stats['efficiency_score'] = 1 / (1 + product_stats['avg_shipping_days'])

        # Composite score
        product_stats['portfolio_score'] = (
            0.4 * product_stats['revenue_per_unit'] / product_stats['revenue_per_unit'].max() +
            0.3 * product_stats['consistency_score'] +
            0.3 * product_stats['efficiency_score']
        )

        top_products = product_stats.nlargest(top_n, 'portfolio_score')

        results = []
        for _, row in top_products.iterrows():
            product_name = self.le.inverse_transform([int(row['product'])])[0]
            results.append({
                'product_id': int(row['product']),
                'product_name': product_name,
                'portfolio_score': row['portfolio_score'],
                'total_sales': row['total_sales'],
                'avg_qty': row['avg_qty']
            })

        print(f"✓ Portfolio optimized with {len(results)} products")
        return results


In [ ]:
class DemandCorrelationAnalyzer:
    """Feature 3: Cross-Product Demand Correlation Analysis"""
    def __init__(self, df):
        self.df = df

    def analyze_correlations(self, top_n=10):
        print(f"  Analyzing correlations for top {top_n} products...")

        top_products = self.df['product'].value_counts().head(top_n).index

        daily_demand = self.df[self.df['product'].isin(top_products)].pivot_table(
            index='order date (DateOrders)',
            columns='product',
            values='Order Item Quantity',
            aggfunc='sum'
        ).fillna(0)

        correlation_matrix = daily_demand.corr()

        correlations = []
        for i in range(len(correlation_matrix)):
            for j in range(i+1, len(correlation_matrix)):
                correlations.append({
                    'product1': correlation_matrix.index[i],
                    'product2': correlation_matrix.columns[j],
                    'correlation': correlation_matrix.iloc[i, j]
                })

        correlations_df = pd.DataFrame(correlations)
        correlations_df = correlations_df.sort_values('correlation', ascending=False)

        print(f"  ✓ Found {len(correlations_df)} product correlations")
        return correlations_df.head(5), correlation_matrix



In [ ]:
class VisualizationDashboard:
    """Feature 6: Advanced Visualization Dashboard"""
    @staticmethod
    def create_dashboard(df, portfolio_results, anomaly_df):
        print("  Creating dashboard...")

        fig = plt.figure(figsize=(16, 10))
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

        # 1. Order Quantity Distribution
        ax1 = fig.add_subplot(gs[0, 0])
        ax1.hist(df['Order Item Quantity'], bins=50, color='skyblue', edgecolor='black')
        ax1.set_title('Order Quantity Distribution', fontweight='bold')
        ax1.set_xlabel('Quantity')
        ax1.set_ylabel('Frequency')

        # 2. Top Products by Sales
        ax2 = fig.add_subplot(gs[0, 1])
        top_sales = df.groupby('Product Name')['Sales'].sum().nlargest(8)
        ax2.barh(range(len(top_sales)), top_sales.values, color='coral')
        ax2.set_yticks(range(len(top_sales)))
        ax2.set_yticklabels([name[:25] + '...' if len(name) > 25 else name for name in top_sales.index], fontsize=8)
        ax2.set_title('Top 8 Products by Sales', fontweight='bold')
        ax2.set_xlabel('Total Sales ($)')

        # 3. Shipping Mode Pie Chart
        ax3 = fig.add_subplot(gs[0, 2])
        shipping_dist = df['Shipping Mode'].value_counts()
        colors = plt.cm.Set3(range(len(shipping_dist)))
        ax3.pie(shipping_dist.values, labels=shipping_dist.index, autopct='%1.1f%%',
               startangle=90, colors=colors)
        ax3.set_title('Shipping Mode Distribution', fontweight='bold')

        # 4. Anomaly Detection Scatter
        ax4 = fig.add_subplot(gs[1, 0])
        if 'is_anomaly' in anomaly_df.columns:
            normal = anomaly_df[anomaly_df['is_anomaly'] == False]
            anomaly = anomaly_df[anomaly_df['is_anomaly'] == True]
            ax4.scatter(normal['Sales'], normal['Order Item Quantity'],
                       alpha=0.3, s=5, label='Normal', color='blue')
            ax4.scatter(anomaly['Sales'], anomaly['Order Item Quantity'],
                       alpha=0.8, s=30, label='Anomaly', color='red', marker='x')
            ax4.set_title('Anomaly Detection Results', fontweight='bold')
            ax4.set_xlabel('Sales ($)')
            ax4.set_ylabel('Order Quantity')
            ax4.legend()

        # 5. Portfolio Scores
        ax5 = fig.add_subplot(gs[1, 1])
        if portfolio_results:
            portfolio_df = pd.DataFrame(portfolio_results)
            colors_portfolio = plt.cm.Greens(np.linspace(0.4, 0.8, len(portfolio_df)))
            ax5.barh(range(len(portfolio_df)), portfolio_df['portfolio_score'], color=colors_portfolio)
            ax5.set_yticks(range(len(portfolio_df)))
            ax5.set_yticklabels([name[:20] + '...' if len(name) > 20 else name
                                for name in portfolio_df['product_name']], fontsize=8)
            ax5.set_title('Portfolio Optimization Scores', fontweight='bold')
            ax5.set_xlabel('Score')

        # 6. Emissions by Mode
        ax6 = fig.add_subplot(gs[1, 2])
        emissions_rates = {'Standard Shipping': 0.2, 'Second Class': 0.25,
                          'First Class': 0.3, 'Same Day': 0.35}
        emissions_by_mode = {}
        for mode, rate in emissions_rates.items():
            qty = df[df['Shipping Mode'] == mode]['Order Item Quantity'].sum()
            emissions_by_mode[mode] = qty * rate

        ax6.bar(range(len(emissions_by_mode)), list(emissions_by_mode.values()),
               color='orange', edgecolor='black')
        ax6.set_xticks(range(len(emissions_by_mode)))
        ax6.set_xticklabels([m.replace(' ', '\n') for m in emissions_by_mode.keys()],
                            fontsize=8, rotation=0)
        ax6.set_title('Emissions by Shipping Mode', fontweight='bold')
        ax6.set_ylabel('Emissions (kg CO2)')

        # 7. Sales Trend Over Time
        ax7 = fig.add_subplot(gs[2, :2])
        monthly_sales = df.groupby(df['order date (DateOrders)'].dt.to_period('M'))['Sales'].sum()
        ax7.plot(range(len(monthly_sales)), monthly_sales.values,
                marker='o', linewidth=2, color='darkgreen')
        ax7.set_title('Monthly Sales Trend', fontweight='bold')
        ax7.set_xlabel('Month Index')
        ax7.set_ylabel('Total Sales ($)')
        ax7.grid(True, alpha=0.3)

        # 8. Demand Heatmap (simplified)
        ax8 = fig.add_subplot(gs[2, 2])
        top_5_products = df['product'].value_counts().head(5).index
        heatmap_data = df[df['product'].isin(top_5_products)].groupby(
            ['product', df['order date (DateOrders)'].dt.dayofweek]
        )['Order Item Quantity'].sum().unstack(fill_value=0)

        if not heatmap_data.empty:
            sns.heatmap(heatmap_data, cmap='YlOrRd', ax=ax8, cbar_kws={'label': 'Quantity'})
            ax8.set_title('Demand by Day of Week', fontweight='bold')
            ax8.set_xlabel('Day of Week')
            ax8.set_ylabel('Product ID')

        plt.suptitle('DASCO Enhanced Supply Chain Analytics Dashboard',
                    fontsize=18, fontweight='bold', y=0.995)

        plt.savefig('dasco_dashboard.png', dpi=300, bbox_inches='tight')
        print("  ✓ Dashboard saved: dasco_dashboard.png")
        plt.show()